# This tutorials walks you through the process of creating new MXNet operators(or layers).

In [1]:
# Custom Op
import os
import mxnet as mx
import numpy as np

/home/poodar/Library/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
class Softmax(mx.operator.CustomOp):
    def forward(self, is_train, req, in_data, out_data, aux):
        x = in_data[0].asnumpy()
        y = np.exp(x-x.max(axis=1).reshape((x.shape[0], 1)))
        y /= y.sum(axis=1).reshape((x.shape[0], 1))
        # At the end, we used CustomOp.assign to assign the resulting array y to out_data[0]. 
        # It handles assignment based on the value of req, which can be ‘write’, ‘add’, or ‘null’.
        self.assign(out_data[0], req[0], mx.nd.array(y))
        
    def backard(self, req, out_grad, in_data, out_data, in_grad, aux):
        l = in_data[1].asnumpy().ravel().astype(np.int)
        y = out_data[0].asnumpy()
        y[np.arange(l.shape[0]), l] -= 1.0
        self.assign(in_grad[0], req[0], mx.nd.array(y))

In [4]:
# Still need to define its input/output format by subclassing mx.operator.CustomOpProp. 
# First, register the new operator with the name ‘softmax’:
@mx.operator.register("softmax")
class SoftmaxProp(mx.operator.CustomOpProp):
    def __init__(self):
        super(SoftmaxProp, self).__init__(need_top_grad=False)
        
    def list_arguments(self):
        return ['data', 'label']

    def list_outputs(self):
        return ['output']
    
    # provide infer_shape to declare the shape of the output/weight 
    # and check the consistency of the input shapes
    def infer_shape(self, in_shape):
        data_shape = in_shape[0]
        label_shape = (in_shape[0][0],)
        output_shape = in_shape[0]
        # The infer_shape function should always return three lists in this order: 
        # inputs, outputs, and auxiliary states 
        return [data_shape, label_shape], [output_shape], []
    
    def infer_type(self, in_type):
        dtype = in_type[0]
        return [dtype, dtype], [dtype], []

    # Finally define a create_operator function that will be calle by the back end to create an instance of softmax:
    def create_operator(self, ctx, shapes, dtypes):
        return Softmax()

In [8]:
# To use the Custom operator:
# define mlp
data = mx.symbol.Variable('data')

fc1 = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=128)
act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2 = mx.symbol.FullyConnected(data = act1, name = 'fc2', num_hidden = 64)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")
fc3 = mx.symbol.FullyConnected(data = act2, name='fc3', num_hidden=10)
#mlp = mx.symbol.Softmax(data = fc3, name = 'softmax')
mlp = mx.symbol.Custom(data=fc3, name='softmax1', op_type='softmax') # op_type 对应前面　ｒｅｇｉｓｔｅｒ　的　ｏｐ　名称

In [9]:
import logging

In [11]:
train, val = get_mnist_iterator(batch_size=100, input_shape=(784, ))
logging.basicConfig(level=logging.DEBUG)

# MXNET_CPU_WORKER_NTHREADS must be greater than 1 for custom op to work on CPU
context = mx.cpu()

mod = mx.mod.Module(mlp, context=context)

mod.fit(train_data=train, 
        eval_data=val, 
        optimizer='sgd', 
        optimizer_params={'learning_rate':0.1, 'momentum':0.9, 'wd':0.00001},
        num_epoch=10, 
        batch_end_callback=mx.callback.Speedometer(100, 100)
       )

NameError: name 'get_mnist_iterator' is not defined